In [2]:
import numpy as np
import pandas as pd

In [3]:
batch_size = 64  
epochs = 100  
latent_dim = 256  # Latent dimensionality of the encoding space.(neurons)
num_samples = 10000  # Number of samples to train on.
data_path = 'fra.txt'

In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[:min(num_samples, len(lines) -1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [5]:
len(input_texts)

10000

In [6]:
len(target_characters)

91

In [7]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [8]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 91
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [9]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)])

In [10]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens))
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens))
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [11]:
# input_token_index

In [12]:
encoder_input_data.shape

(10000, 14, 70)

In [13]:
# one hot encoding, where ever the char is present replacing there with 1.
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1. # last character space (acts as padding)

    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # target output is shifted by one step for teacher forcing.(layers)
            #Input at time t should predict the next character (t-1).
            decoder_target_data[i, t-1, target_token_index[char]] = 1.
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [16]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [17]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, None, 70)          │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_2 (InputLayer)    │ (None, None, 91)          │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 256), (None,      │         334,848 │ input_layer[0][0]          │
│                               │ 256), (None, 256)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_2 (LSTM)                 │ [(None, None, 256),       │         356,352 │ input_layer_2[0][0],       │
│                               │ (None, 256), (None, 256)] │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, None, 91)          │          23,387 │ lstm_2[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 714,587 (2.73 MB)

 Trainable params: 714,587 (2.73 MB)

 Non-trainable params: 0 (0.00 B)

In [92]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [93]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
          batch_size=batch_size, epochs = epochs, validation_split = 0.2)

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 64s 443ms/step - accuracy: 0.7075 - loss: 1.5679 - val_accuracy: 0.7142 - val_loss: 1.0743
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 57s 454ms/step - accuracy: 0.7473 - loss: 0.9577 - val_accuracy: 0.7214 - val_loss: 0.9873
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 57s 456ms/step - accuracy: 0.7606 - loss: 0.8638 - val_accuracy: 0.7513 - val_loss: 0.8831
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 85s 477ms/step - accuracy: 0.7825 - loss: 0.7758 - val_accuracy: 0.7763 - val_loss: 0.7794
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 57s 456ms/step - accuracy: 0.8046 - loss: 0.6819 - val_accuracy: 0.7931 - val_loss: 0.7142
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 54s 435ms/step - accuracy: 0.8151 - loss: 0.6383 - val_accuracy: 0.8013 - val_loss: 0.6853
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 55s 443ms/step - accuracy: 0.8222 - loss: 0.6101 - val_accuracy: 0.8067 - val_loss: 0.6558
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 54s 428ms/step - accuracy: 0.8296 -

In [95]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, None, 70)          │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_3 (InputLayer)    │ (None, None, 91)          │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, 256), (None,      │         334,848 │ input_layer_1[0][0]        │
│                               │ 256), (None, 256)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_2 (LSTM)                 │ [(None, None, 256),       │         356,352 │ input_layer_3[0][0],       │
│                               │ (None, 256), (None, 256)] │                 │ lstm_1[0][1], lstm_1[0][2] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, None, 91)          │          23,387 │ lstm_2[1][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,429,176 (5.45 MB)

 Trainable params: 714,587 (2.73 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 714,589 (2.73 MB)

In [96]:
model.save("s2s_model.keras")

In [19]:
import keras
model = keras.models.load_model('s2s_model.keras')

In [26]:
model.layers[4]

<Dense name=dense_1, built=True>

In [28]:
# Encoder model
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

In [32]:
# Decoder Model
decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
                                                decoder_inputs, 
                                                initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs,
                            [decoder_outputs] + decoder_states)


In [33]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [34]:
encoder_model.layers

[<InputLayer name=input_layer_1, built=True>, <LSTM name=lstm_1, built=True>]

In [35]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [36]:
for seq_index in range(40):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Fais un solli.

-
Input sentence: Go.
Decoded sentence: Fais un solli.

-
Input sentence: Go.
Decoded sentence: Fais un solli.

-
Input sentence: Go.
Decoded sentence: Fais un solli.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run!
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded